## Parallel Python -- concurrency

The `concurrent.futures` library can be used to submit parallel jobs running either on separate threads or processors. Because as we'll see next  in the next notebook the `multiprocessing` library has a different syntax for doing multiprocessing, this library is more commonly used for multithreading, i.e., running concurrent jobs on the same process. 


In [1]:
from concurrent.futures import ThreadPoolExecutor
import requests 
import time

### A function to be parallelized
We want to make a number of google searches. Here we use `requests` to make a query to google using the RESTful API URL string. 

In [2]:
def gsearch(query):
    res = requests.get("http://google.com/search?q={}".format(query))
    return res.content

### Time execution to perform five searches on 1 vs 4 threads. 

In [3]:
queries = [
    "dog", "cat", "mouse", "bird", "mushroom",
    "weasel", "elephant", "rat", "tree",
]

In [4]:
%%timeit -n1 -r1
with ThreadPoolExecutor(max_workers=4) as executor:
    
    # submit queries to threads
    jobs = [executor.submit(gsearch, q) for q in queries]
    
    # collect results
    results = [i.result() for i in jobs]

2.81 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [5]:
%%timeit -n1 -r1
with ThreadPoolExecutor(max_workers=1) as executor:
    
    # submit queries to threads
    jobs = [executor.submit(gsearch, q) for q in queries]
    
    # collect results
    results = [i.result() for i in jobs]

8.06 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


### Explained
The `with` context manager ensures that the ThreadPool is properly shutdown if the jobs are interrupted for any reason. By initiating a ThreadPoolExecutor object with some number of workers we can then start to send jobs to those workers. This is done using the `submit` function call, which returns an asynchronous result object. This object can be used to retrieve the result of the function once its finished. This is done by making the `result()` request later. 

## A test

In [6]:
import numpy as np
a = np.arange(1000000)
b = np.arange(1000000)
c = np.arange(1000000)
d = np.arange(1000000)

### 1. Calculate the sum of all values in a,b,c and d.
Use `timeit` to measure how long it takes to do this without parallel code.

In [7]:
%%timeit
np.sum(a + b + c + d)

3.92 ms ± 327 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### 2. Calculate the sum of all values in a,b,c and d
Use `timeit` to measure how long it takes to do this *with* parallel code. Clue: call `executor.submit(np.sum, arr)` for each array, and then sum the results. 

In [8]:
%%timeit
with ThreadPoolExecutor(max_workers=4) as executor:
    
    # submit queries to threads
    jobs = [executor.submit(np.sum, i) for i in (a, b, c, d)]
    
    # collect results
    res = sum(i.result() for i in jobs)

1.83 ms ± 59.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
